<a href="https://colab.research.google.com/github/KelvinLam05/Zero-Shot-Text-Classification/blob/main/Zero_Shot_Text_Classification_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Zero-Shot Text Classification**

In zero-shot text classification, the model can classify any text between given labels without any prior data.

**Goal of the project**

Let’s build a zero-shot text classifier of Scotch whiskies labeled *Single Malt Scotch*, *Blended Scotch Whisky* and *Blended Malt Scotch Whisky*.

In [20]:
# Importing libraries
import pandas as pd
import numpy as np
import ktrain
import tensorflow as tf
from ktrain import text
from sklearn.model_selection import train_test_split

In [21]:
# Load dataset
df = pd.read_csv('/content/scotch_whisky_review.csv')

In [22]:
# Examine the data
df.head()

,name,category,review.point,price,currency,description.1.2247.
0,Caol Ila 18 year old (Diageo Special Releases ...,Single Malt Scotch,88,100,$,"If you like sherried malts, you’ll love this! ..."
1,"Carlyle, 40%",Single Malt Scotch,88,13,$,"A dense, suffocating fog of peat smoke, sea sa..."
2,"Big Peat Christmas Edition 2017, 54.1%",Blended Malt Scotch Whisky,88,70,$,The style and class of the youngest was inspir...
3,"J. Mossman Gold Crown 12 year old, 40%",Blended Scotch Whisky,88,43,$,"After six ‘Work in Progress’ releases, Kilkerr..."
4,Chapter 7 2008 (distilled at Allt-a-Bhainne) 9...,Single Malt Scotch,88,65,$,"As the name implies, this non-chill filtered e..."


In [23]:
# Overview of all variables, their datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 1000 non-null   object
 1   category             1000 non-null   object
 2   review.point         1000 non-null   int64 
 3   price                1000 non-null   object
 4   currency             1000 non-null   object
 5   description.1.2247.  983 non-null    object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


**Preprocessing**

In [24]:
# Rename column header
df.rename(columns = {'description.1.2247.': 'description'}, inplace = True)

In [25]:
# Checking for missing values
df.isnull().sum().sort_values(ascending = False)

description     17
currency         0
price            0
review.point     0
category         0
name             0
dtype: int64

In [26]:
# Drop rows with NaN values
df = df[pd.notnull(df['description'])]

In [27]:
df.isnull().sum().sort_values(ascending = False)

description     0
currency        0
price           0
review.point    0
category        0
name            0
dtype: int64

In [28]:
# Drop columns that are not needed
df = df[['description', 'category']]

In [29]:
# Checking the distribution of classes
df['category'].value_counts() 

Single Malt Scotch            830
Blended Scotch Whisky          96
Blended Malt Scotch Whisky     57
Name: category, dtype: int64

We have a highly unbalanced dataset.

In [30]:
# Find all unique characters and symbols 
all_text = str()

for sentence in df['description'].values:
    all_text += sentence
    
''.join(set(all_text))

'\n.)x6€9;âlmô XQ(MJT‘-8GgdpC\rZ5&U—é£/bsWh?nyàR0…D’K\'q4òVrN\xa0P:32zB%1OvcaüF”S–fùwAkè#!e“$ÌtYHoEI"Lui,7j\u2028ûç'

The kind of data we get from customer feedback is usually unstructured. It contains unusual text and symbols that need to be cleaned so that a machine learning model can grasp it.

In [31]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

In [32]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
stop_words = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

We will now set up our cleaning function.

In [34]:
def clean_review(review_text):

  # Removing all irrelevant characters (numbers and punctuation)                           
  review_text = re.sub('[^a-zA-Z]', ' ', review_text)                           
  # Replace one or more spaces with single space
  review_text = re.sub(r'\s+', ' ', review_text)                                
  # Convert all characters into lowercase
  review_text = str(review_text).lower()                                        
  # Tokenization
  review_text = word_tokenize(review_text)
  # Removing Stopwords                                      
  review_text = [item for item in review_text if item not in stop_words]        
  # Lemmatization
  review_text = [lemma.lemmatize(word = w, pos = 'v') for w in review_text]     
  # Remove the words having length <= 2
  review_text = [i for i in review_text if len(i) > 2]                          
  # Convert the list of tokens into back to the string
  review_text = ' '.join (review_text)                                          
  
  return review_text 

In [35]:
df['clean_review'] = df['description'].apply(clean_review)

In [36]:
all_text = str()

for sentence in df['clean_review'].values:
    all_text += sentence
    
''.join(set(all_text))

'wkxrgdplmebt szhonyuivcajqf'

In [37]:
pip install pyspellchecker

     |████████████████████████████████| 2.7 MB 3.6 MB/s 


In [38]:
from spellchecker import SpellChecker

In [39]:
# Instantiate spell checker
spell = SpellChecker()

In [40]:
# Correct spelling
def correct_spellings(text):
    
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    
    return ' '.join(corrected_text)

In [41]:
df['clean_review'] = df['clean_review'].apply(correct_spellings)

In [42]:
# Get the first five rows
df['description'].head()

0    If you like sherried malts, you’ll love this! ...
1    A dense, suffocating fog of peat smoke, sea sa...
2    The style and class of the youngest was inspir...
3    After six ‘Work in Progress’ releases, Kilkerr...
4    As the name implies, this non-chill filtered e...
Name: description, dtype: object

In [43]:
# Get the first five rows
df['clean_review'].head()

0    like sherries malt love bottle respectable str...
1    dense suffocate fog peat smoke sea salt dry se...
2    style class youngest inspire classic elegance ...
3    six work progress release kilkerran glendale d...
4    name imply non chill filter expression finish ...
Name: clean_review, dtype: object

In [44]:
# Display full strings
with pd.option_context('display.max_colwidth', None):
  display(df['clean_review'])

0                                                                                                                 like sherries malt love bottle respectable strength red apple cherry skin strawberry raspberry eccles cake malt loaf warm spice lot get nose finely structure dram soft leather rhubarb bradley apple cherryade fresh victoria plum pepper mute ginger deliver sustain flavor long spicy peel fruit finish give distillery closure could interest components bottle
1                                                                                                                                                       dense suffocate fog peat smoke sea salt dry seaweed high tide lemon scent candle remember come air supple silky texture lemon mousse bake apple vanilla cinnamon ginger massive rush pepper hold mouth long possible flavor delivery impressively long constantly evolve hot dry finish frankly relief peppery assault palate
2                                                           

**Testing for GPU**

In [45]:
import torch

In [46]:
torch.cuda.is_available()

True

In [47]:
device = torch.cuda.current_device() if torch.cuda.is_available() else -1

In [48]:
print(device)

0


In [49]:
# Load
Load_on_CPU = torch.device('cuda')

**Transformer Pipeline**

We’ll use the appropriate transformers.pipeline to compute the predicted class for each Scotch whisky.

In [50]:
from transformers import pipeline

In [51]:
task = 'zero-shot-classification'
zero_shot_model = 'vicgalle/xlm-roberta-large-xnli-anli'
zero_shot_classifier = pipeline(task, zero_shot_model, device = device)

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [52]:
sequences = 'like sherried malt love bottle respectable strength red apple cherry skin strawberry raspberry eccles cake malt loaf warm spice lot get nose finely structure dram soft leather rhubarb bramley apple cherryade fresh victoria plum pepper mute ginger deliver sustain flavor long spicy peel fruit finish give distillery closure could interest components bottle'

In [53]:
candidate_labels = ['Single Malt Scotch', 'Blended Scotch Whisky', 'Blended Malt Scotch Whisky']

In [54]:
outputs = zero_shot_classifier(sequences = sequences, candidate_labels = candidate_labels, multi_class = False)

Let’s take a look at the outputs.

In [55]:
for label, score in zip(outputs['labels'], outputs['scores']):
    print(f'{label}: {score:.3f}')

Single Malt Scotch: 0.473
Blended Scotch Whisky: 0.267
Blended Malt Scotch Whisky: 0.260


The model correctly identifies that the likely label is Single Malt Scotch. Other irrelevant labels, such as Blended Scotch Whisky and Blended Malt Scotch Whisky, have a very low score.

In [56]:
task = 'zero-shot-classification'
zero_shot_model = 'vicgalle/xlm-roberta-large-xnli-anli'
classifier = pipeline(task, zero_shot_model, device = device) 

In [57]:
candidate_labels = ['Single Malt Scotch', 'Blended Scotch Whisky', 'Blended Malt Scotch Whisky']

In [58]:
# Compute the predicted label for each whisky
df['label_pred_zero_shot'] = df['clean_review'].apply(lambda x: classifier(x, candidate_labels = candidate_labels)['labels'][0])

In [59]:
from sklearn.metrics import accuracy_score

In [60]:
# Accuracy
accuracy_score(df['category'], df['label_pred_zero_shot'])

0.7019328585961343

The Zero-Shot classifier does a decent job. Glancing at a few random reviews uncorrectly labeled by the Zero-Shot classifier, there does not seem to be a particularly problematic class, although such a assertion would require further investigation. But the length of the review could lead to poor performance. We can read about this on the Hugging Face forum. Joe Davison, Hugging Face developer and creator of the Zero-Shot pipeline, says the following:

*For long documents, I don’t think there’s an ideal solution right now. If truncation isn’t satisfactory, then the best thing you can do is probably split the document into smaller segments and ensemble the scores somehow.*

We’ll try another solution: summarizing the article first, then Zero-Shot classifying it.

**Initialize and summarizing with Bert Summarizer + Zero-Shot Classification**

In [61]:
# Find the length of strings
df['clean_review_length'] = df['clean_review'].apply(len)

In [62]:
# Generate descriptive statistics 
df['clean_review_length'].describe()

count    983.000000
mean     285.380468
std       63.673906
min      116.000000
25%      249.500000
50%      286.000000
75%      320.000000
max      667.000000
Name: clean_review_length, dtype: float64

In [63]:
# Find the longest string
min(df['clean_review'], key = len)

'lemonade hint aniseed putty nose tropical fruit spice milk chocolate palate finish medium length spicy hint licorice'

In [64]:
# Find the shortest string
max(df['clean_review'], key = len)

'color antique gold aroma dry creamy note vanilla marshmallow honey tropical fruit pineapple coconut palate malty creamy front vanilla marshmallow hint honey briefly become fruity tropical fruit turn dry oaky big long dry finish especially triple distil lowlands lowland whiskies know mature nicely younger age people know especially auchentoshan delicious older age mention still column older vintages auchentoshan offer individual retailers one cask time one casks one many vintage auchentoshan whiskies enjoy past years auchentoshan balance complexity perspective seem best year old range still say one hold fairly well age available exclusively park avenue liquors'

**Text Summarization with BERT**

To see how the summarization model works, we’ll do a quick dry run. 



In [65]:
from transformers import BertTokenizerFast, EncoderDecoderModel

In [66]:
review_text = max(df['clean_review'], key = len)

In [67]:
summarization_model = 'mrm8488/bert-small2bert-small-finetuned-cnn_daily_mail-summarization'

In [68]:
model = EncoderDecoderModel.from_pretrained(summarization_model).to(Load_on_CPU)
tokenizer = BertTokenizerFast.from_pretrained(summarization_model)

inputs = tokenizer(review_text, padding = 'max_length', truncation = True, max_length = 512, return_tensors = 'pt')
input_ids = inputs.input_ids.to(Load_on_CPU)
attention_mask = inputs.attention_mask.to(Load_on_CPU)

output = model.generate(input_ids, attention_mask = attention_mask, max_length = 286, no_repeat_ngram_size = 2, num_beams = 4, do_sample = False, early_stopping = True)
summary_text = tokenizer.decode(output[0], skip_special_tokens = True)

print(summary_text)

Downloading:   0%|          | 0.00/3.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/247M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/324 [00:00<?, ?B/s]

the color antique gold aroma dry creamy note vanilla marshmallow honey tropical fruit turned dry oaky big long dry finish especially triple distil lowlands lowland whiskies know mature nicely younger age people know especially auchentoshan delicious older age mention still column older vintages auchnoshan offer individual retailers one cask time one.


**Initialize and summarizing with Bert Summarizer + Zero-Shot Classification**

In [69]:
model = EncoderDecoderModel.from_pretrained(summarization_model).to(Load_on_CPU)
tokenizer = BertTokenizerFast.from_pretrained(summarization_model)

# Custom summarization pipeline (to handle long reviews)
def generate_summary(text):
    
    inputs = tokenizer(text, padding = 'max_length', truncation = True, max_length = 512, return_tensors = 'pt')
    input_ids = inputs.input_ids.to(Load_on_CPU)
    attention_mask = inputs.attention_mask.to(Load_on_CPU)

    output = model.generate(input_ids, attention_mask = attention_mask, max_length = 286, no_repeat_ngram_size = 2, num_beams = 4, do_sample = False, early_stopping = True)
    
    return tokenizer.decode(output[0], skip_special_tokens = True)

In [70]:
task = 'zero-shot-classification'
zero_shot_model = 'vicgalle/xlm-roberta-large-xnli-anli'
classifier = pipeline(task, zero_shot_model, device = device) 

In [71]:
candidate_labels = ['Single Malt Scotch', 'Blended Scotch Whisky', 'Blended Malt Scotch Whisky']

In [72]:
# Apply summarization then zero-shot classification to the dataset
df['label_pred_bert-small_sum_zs'] = df['clean_review'].apply(lambda x: classifier(generate_summary(x), candidate_labels = candidate_labels, multi_class = False)['labels'][0])

In [73]:
# Accuracy
accuracy_score(df['category'], df['label_pred_bert-small_sum_zs'])

0.7375381485249237

Adding the summarization before the zero-shot classification, **the accuracy jumped by ~3.6%**.

**Text Summarization with T5**

To see how the summarization model works, we’ll do a quick dry run.

In [74]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [75]:
review_text = max(df['clean_review'], key = len)

In [76]:
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(Load_on_CPU)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

input_ids = tokenizer.encode(review_text, padding = 'max_length', truncation = True, max_length = 512, return_tensors = 'pt').to(Load_on_CPU)
summary_ids = model.generate(input_ids, max_length = 286, no_repeat_ngram_size = 2, num_beams = 4, do_sample = False, early_stopping = True)

summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens = True)

print(summary_text)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

gold aroma dry creamy note vanilla marshmallow honey tropical fruit pineapple coconut palate malty creamy front vanilla  marshmallow hint honey briefly become fruity tropical fruits turn dry oaky big long dry finish especially triple distil lowlands lowland whiskies know mature nicely younger age people know especially auchentoshan delicious older age mention still column older vintages Auchentah offer individual retailers one cask time one cazks one many vintage aussienthodote whiskeys enjoy past years auchethyl sanita


**Initialize and summarizing with T5 Summarizer + Zero-Shot Classification**

In [77]:
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(Load_on_CPU)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Custom summarization pipeline (to handle long reviews)
def generate_summary(text):

    input_ids = tokenizer.encode(text, padding = 'max_length', truncation = True, max_length = 512, return_tensors = 'pt').to(Load_on_CPU)
    summary_ids = model.generate(input_ids, max_length = 286, no_repeat_ngram_size = 2, num_beams = 4, do_sample = False, early_stopping = True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens = True)

In [78]:
task = 'zero-shot-classification'
zero_shot_model = 'vicgalle/xlm-roberta-large-xnli-anli'
classifier = pipeline(task, zero_shot_model, device = device) 

In [79]:
candidate_labels = ['Single Malt Scotch', 'Blended Scotch Whisky', 'Blended Malt Scotch Whisky']

In [80]:
# Apply summarization then zero-shot classification to the dataset
df['label_pred_t5-small_sum_zs'] = df['clean_review'].apply(lambda x: classifier(generate_summary(x), candidate_labels = candidate_labels, multi_class = False)['labels'][0])

In [81]:
# Accuracy
accuracy_score(df['category'], df['label_pred_t5-small_sum_zs'])

0.7263479145473042

Adding the summarization before the zero-shot classification, **the accuracy jumped by ~2.4%**.